In [1]:
import math
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import xgboost as xgb
import pickle

In [2]:
# Read and concat the data
PL_20_21 = pd.read_csv('data\\PL_20_21.csv', sep=',')
PL_19_20 = pd.read_csv('data\\PL_19_20.csv', sep=',')
PL_18_19 = pd.read_csv('data\\PL_18_19.csv', sep=',')
PL_17_18 = pd.read_csv('data\\PL_17_18.csv', sep=',')
PL_16_17 = pd.read_csv('data\\PL_16_17.csv', sep=',')
BL_20_21 = pd.read_csv('data\\BL_20_21.csv', sep=',')
BL_19_20 = pd.read_csv('data\\BL_19_20.csv', sep=',')
BL_18_19 = pd.read_csv('data\\BL_18_19.csv', sep=',')
BL_17_18 = pd.read_csv('data\\BL_17_18.csv', sep=',')
BL_16_17 = pd.read_csv('data\\BL_16_17.csv', sep=',')
SP_20_21 = pd.read_csv('data\\SP_20_21.csv', sep=',')
SP_19_20 = pd.read_csv('data\\SP_19_20.csv', sep=',')
SP_18_19 = pd.read_csv('data\\SP_18_19.csv', sep=',')
SP_17_18 = pd.read_csv('data\\SP_17_18.csv', sep=',')
SP_16_17 = pd.read_csv('data\\SP_16_17.csv', sep=',')
IT_20_21 = pd.read_csv('data\\IT_20_21.csv', sep=',')
IT_19_20 = pd.read_csv('data\\IT_19_20.csv', sep=',')
IT_18_19 = pd.read_csv('data\\IT_18_19.csv', sep=',')
IT_17_18 = pd.read_csv('data\\IT_17_18.csv', sep=',')
IT_16_17 = pd.read_csv('data\\IT_16_17.csv', sep=',')

data = pd.concat([PL_20_21,PL_19_20,PL_18_19,PL_17_18,PL_16_17,
                  BL_20_21,BL_19_20,BL_18_19,BL_17_18,BL_16_17,
                  SP_20_21,SP_19_20,SP_18_19,SP_17_18,SP_16_17,
                  IT_20_21,IT_19_20,IT_18_19,IT_17_18,IT_16_17])
data = data.reset_index(drop=True)
data

,Date,HomeTeam,AwayTeam,FTR,h_course,d_course,a_course,h_pts_avg3,h_pts_avg5,h_pts_avg7,...,a_rc_min7,a_pts_per_math,a_gz,a_gs,a_sh_od,a_sh_ot,a_cw,a_cb,a_pozycja,position_dst
0,2020-09-12,Crystal Palace,Southampton,H,3.10,3.25,2.37,1.000000,1.2,0.857143,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-09-12,Liverpool,Leeds,H,1.28,6.00,9.50,3.000000,3.0,2.428571,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-09-12,West Ham,Newcastle,A,2.15,3.40,3.40,2.333333,2.0,1.428571,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-09-13,West Brom,Leicester,A,3.80,3.60,1.95,0.000000,0.2,0.285714,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-09-13,Tottenham,Everton,A,1.83,3.60,4.33,2.000000,1.8,1.857143,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7205,2017-05-28,Inter,Udinese,H,1.33,5.50,8.50,2.000000,1.2,1.000000,...,0,1.216216,1.216216,1.378378,11.270270,12.297297,5.540541,4.945946,13.0,5.0
7206,2017-05-28,Palermo,Empoli,H,6.00,4.33,1.53,2.000000,2.0,1.571429,...,2,0.864865,0.756757,1.594595,9.324324,16.054054,5.702703,4.027027,18.0,1.0
7207,2017-05-28,Roma,Genoa,H,1.06,15.00,34.00,3.000000,2.4,2.285714,...,0,0.972973,0.972973,1.648649,11.621622,12.270270,4.540541,5.216216,16.0,14.0
7208,2017-05-28,Sampdoria,Napoli,A,15.00,7.50,1.18,0.666667,0.6,0.428571,...,0,2.243243,2.432432,1.000000,17.513514,10.027027,6.162162,4.540541,3.0,7.0


In [3]:
# Dictionary for mapping target
str2idx = {'H':0, 'D':1, 'A':2}
idx2str = {0:'H', 1:'D', 2:'A'}
dicts2translate = {'str2idx' : str2idx,
                   'idx2str' : idx2str }

courses = data[['h_course','d_course','a_course','FTR']]

data = data.replace({'FTR': str2idx})

data_label = data['FTR']
data = data.drop(['FTR','Date','HomeTeam','AwayTeam'], axis=1)
data

,h_course,d_course,a_course,h_pts_avg3,h_pts_avg5,h_pts_avg7,h_pts_std3,h_pts_std5,h_pts_std7,h_pts_wz3,...,a_rc_min7,a_pts_per_math,a_gz,a_gs,a_sh_od,a_sh_ot,a_cw,a_cb,a_pozycja,position_dst
0,3.10,3.25,2.37,1.000000,1.2,0.857143,1.414214,1.469694,1.355262,1.414214,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.28,6.00,9.50,3.000000,3.0,2.428571,0.000000,0.000000,0.903508,0.000000,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.15,3.40,3.40,2.333333,2.0,1.428571,0.942809,1.264911,1.399708,0.404061,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.80,3.60,1.95,0.000000,0.2,0.285714,0.000000,0.400000,0.451754,NaN,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.83,3.60,4.33,2.000000,1.8,1.857143,1.414214,1.469694,1.355262,0.707107,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7205,1.33,5.50,8.50,2.000000,1.2,1.000000,1.414214,1.469694,1.309307,0.707107,...,0,1.216216,1.216216,1.378378,11.270270,12.297297,5.540541,4.945946,13.0,5.0
7206,6.00,4.33,1.53,2.000000,2.0,1.571429,1.414214,1.264911,1.293626,0.707107,...,2,0.864865,0.756757,1.594595,9.324324,16.054054,5.702703,4.027027,18.0,1.0
7207,1.06,15.00,34.00,3.000000,2.4,2.285714,0.000000,1.200000,1.160577,0.000000,...,0,0.972973,0.972973,1.648649,11.621622,12.270270,4.540541,5.216216,16.0,14.0
7208,15.00,7.50,1.18,0.666667,0.6,0.428571,0.471405,0.489898,0.494872,0.707107,...,0,2.243243,2.432432,1.000000,17.513514,10.027027,6.162162,4.540541,3.0,7.0


In [4]:
# Drop odds columns
data = data.drop(['h_course','d_course','a_course'], axis=1)

In [5]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(data, data_label, test_size=0.30, random_state=20)

In [6]:
# Create table with odds only for test data
courses = courses.iloc[X_test.index]
courses

,h_course,d_course,a_course,FTR
3390,8.00,4.75,1.40,A
3624,2.60,3.00,3.00,H
6571,3.39,3.60,2.04,A
4304,1.40,4.75,8.00,H
6970,1.40,4.50,9.00,H
...,...,...,...,...
6465,2.50,3.20,2.88,D
6272,3.75,3.50,2.00,D
4139,4.20,3.20,2.00,D
3097,3.60,3.29,2.10,A


In [7]:
# PREDICTIONS

xgb_model = pickle.load(open('model\\xgb_model.pkl', "rb"))
tree_model = pickle.load(open('model\\tree_model.pkl', "rb"))
dicts2translate = pickle.load(open('model\\dicts2translate.pkl', "rb"))

preds = pd.DataFrame(xgb_model.predict_proba(X_test))
preds = preds.set_index(X_test.index)

courses['pr_h_won'] = preds[0]
courses['pr_draw']  = preds[1]
courses['pr_a_won'] = preds[2]

tree_preds = tree_model.predict(preds)
preds_after_translation = [dicts2translate['idx2str'][elem] for elem in tree_preds]
courses['prediction'] = preds_after_translation
courses = courses.round({'pr_h_won': 4, 'pr_draw': 4, 'pr_a_won': 4})

def get_win_odds(row):
    if row.prediction == 'A':
        return row.a_course
    if row.prediction == 'D':
        return row.d_course
    if row.prediction == 'H':
        return row.h_course

courses['win_odds'] = courses.apply(lambda row: get_win_odds(row), axis=1)
courses

,h_course,d_course,a_course,FTR,pr_h_won,pr_draw,pr_a_won,prediction,win_odds
3390,8.00,4.75,1.40,A,0.1431,0.1780,0.6790,A,1.40
3624,2.60,3.00,3.00,H,0.5064,0.2722,0.2214,H,2.60
6571,3.39,3.60,2.04,A,0.3108,0.2218,0.4674,A,2.04
4304,1.40,4.75,8.00,H,0.5971,0.2163,0.1866,H,1.40
6970,1.40,4.50,9.00,H,0.5680,0.2226,0.2094,H,1.40
...,...,...,...,...,...,...,...,...,...
6465,2.50,3.20,2.88,D,0.5124,0.2287,0.2589,H,2.50
6272,3.75,3.50,2.00,D,0.3400,0.2133,0.4468,A,2.00
4139,4.20,3.20,2.00,D,0.2475,0.3580,0.3944,D,3.20
3097,3.60,3.29,2.10,A,0.2044,0.2281,0.5675,A,2.10


In [8]:
def find_best_odds(df, odds):
    """
    The function returns the ratio of correctly classified matches at a given odds.
    """
    df = df[df['win_odds'] > odds]
    # Accuracy for
    return len(df[df.prediction == df.FTR])/len(df)

In [9]:
# E.g. for 'odds = 2.5' ratio of correctly classified matches is 0.3004484304932735
find_best_odds(df = courses, odds = 2.5)

0.3004484304932735

In [10]:
def calc_income(df, odds, bid = 10, tax = 0.12, n_games = 10, model_adjust = 0):
    """
    The function returns a dataframe with income under the given boundary conditions.
    
        Arguments:
    df - data frame - courses
    odds - a list with the bookmaker odds for which we want to calculate the income
    bid - the value of a single bet. By default 10$.
    tax - tax on winning. Default 12%.
    n_games - number of bets. Otherwise the number of withdrawn matches. Default 10.
    model_adjust - [optional] NOTE: this parameter allows to estimate the income if the accuracy of the model increase by 'model_adjust'
    
        Returns:
    income_list - DataFrame with income under the given boundary conditions
    """
    win_chance_list = list(map(lambda x: find_best_odds(df,x), odds ))
    win_chance_list = [win_chance_list[x] + model_adjust for x in range(len(win_chance_list))]  
    
    income_list = []
    for i in range(len(win_chance_list)):
        one_game_income = bid*odds[i]*(1-tax)-bid
        one_game_loss = bid
        
        n_win_games = (win_chance_list[i]*n_games)
        n_lost_games = ((1-win_chance_list[i])*n_games)
        income = n_win_games*one_game_income - n_lost_games*one_game_loss
        income_list.append(income)
    return pd.DataFrame({'odds':odds,
                         'income':income_list})

In [13]:
# E.g.
# If we only bet on matches with odds > 3.4
# and we increase the quality of the model by 0.05,
# the avg income will be 27$ per 10 matches with bet 100$.

calc_income(df = courses,
            odds = np.arange(2.6,4.1,0.1),
            bid = 100,
            tax = 0.12,
            n_games = 10,
            model_adjust=0.05)

,odds,income
0,2.6,-208.457803
1,2.7,-174.976173
2,2.8,-141.869307
3,2.9,-112.021483
4,3.0,-102.008097
5,3.1,-84.171429
6,3.2,-42.835179
7,3.3,-51.014286
8,3.4,27.019355
9,3.5,27.600000
